In [1]:
from pyspark.sql import *
from pyspark.sql.functions import *
from pyspark import SparkContext
from pyspark import SQLContext

# create the Spark Session
spark = SparkSession.builder.getOrCreate()

# create the Spark Context
sc = spark.sparkContext

In [7]:
rddVentas = sc.textFile('2Parcial2019.csv').map(lambda x: (int(x.split(',')[0]), int(x.split(',')[1]), x.split(',')[2])) #(marca, modelo, fecha(YY-MM-DD))
rddVentas.take(5)

[(1, 3, '19-02-08'),
 (4, 3, '18-06-30'),
 (5, 2, '19-01-02'),
 (5, 2, '19-02-22'),
 (5, 1, '18-08-18')]

In [13]:
# Creo un rdd con la forma ((marca, modelo), (ventas_totales, primera_venta, ultima_venta))
rddVentasModificado = rddVentas.map(lambda x: ((x[0], x[1]), (1, x[2], x[2]))).\
                      reduceByKey(lambda a,b: (a[0]+a[0], a[1] if a[1]<b[1] else b[1], a[2] if a[2]>b[2] else b[2]))
rddVentasModificado.take(20)

[((1, 3), (8, '18-09-13', '19-03-05')),
 ((5, 1), (4, '18-07-13', '19-07-16')),
 ((2, 2), (2, '18-10-21', '19-07-15')),
 ((4, 2), (16, '18-08-04', '19-06-16')),
 ((3, 3), (4, '18-07-09', '19-06-02')),
 ((5, 3), (4, '18-07-01', '19-06-27')),
 ((3, 1), (4, '18-07-19', '19-06-23')),
 ((1, 1), (4, '18-08-26', '19-07-27')),
 ((4, 3), (16, '18-06-30', '18-12-10')),
 ((5, 2), (64, '18-06-19', '19-06-30')),
 ((1, 2), (32, '18-07-31', '19-05-22')),
 ((4, 1), (16, '18-11-03', '19-05-03')),
 ((3, 2), (32, '18-06-24', '19-07-27')),
 ((2, 3), (32, '18-11-05', '19-07-21')),
 ((2, 1), (8, '18-08-01', '19-05-16'))]

In [14]:
rddModelosDescontinuados = rddVentasModificado.filter(lambda x: x[1][2] <= '19-06-13')
rddModelosDescontinuados.take(20)

[((1, 3), (8, '18-09-13', '19-03-05')),
 ((3, 3), (4, '18-07-09', '19-06-02')),
 ((4, 3), (16, '18-06-30', '18-12-10')),
 ((1, 2), (32, '18-07-31', '19-05-22')),
 ((4, 1), (16, '18-11-03', '19-05-03')),
 ((2, 1), (8, '18-08-01', '19-05-16'))]